# Gemma Analytics Hiring Test – Surgeon Skill Ranking

Author: David Gonzalez  
Date: 11.07.2025

This notebook analyzes the performance of surgeons conducting hip replacement operations based on EQ-5D-5L questionnaire results.

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
from sqlalchemy import create_engine
import pandas as pd

# DB Credentials
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Create SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}")


### 1. Explore Available Tables

Explore available tables inside DB to find out which ones might be relevant to us.

High priority tables found:

 + patients
 + surgeons
 + answer_options
 + answers
 + questionnaires
 + questions

 Explore high priority tables to see their column names and how their data types look like. 

In [ ]:
# Look for relevant tables within list of tables
tables_df = pd.read_sql("""
    SELECT table_name
    FROM information_schema.tables
    ORDER BY table_name;
""", engine)

with pd.option_context('display.max_rows', None):
    display(tables_df)

In [ ]:
# Preview tables

pd.read_sql("SELECT * FROM answers LIMIT 1;", engine)

In [ ]:
pd.read_sql("SELECT * FROM answer_options LIMIT 1;", engine)

In [ ]:
pd.read_sql("SELECT * FROM questionnaires LIMIT 1;", engine)

In [ ]:
pd.read_sql("SELECT * FROM questions LIMIT 1;", engine)

In [ ]:
pd.read_sql("SELECT * FROM patients LIMIT 1;", engine)

In [ ]:
pd.read_sql("SELECT * FROM surgeons LIMIT 1;", engine)

### 2. Extracting Patient Responses with Health Scores

SQL queries are saved in the `/sql` folder and loaded in this notebook.

This query joins:
- `answers` → raw responses
- `answer_options` → to get `central_estimate` scores
- `questionnaires` → to filter only `Hip` operations and distinguish `pre/post`

The result is one row per question answered, with the corresponding health score component.


In [ ]:
with open("../sql/responses_and_scores.sql", "r") as file:
    query = file.read()

patient_scores_df = pd.read_sql(query, engine)
patient_scores_df.head(3)

### 3. Pre - Post scores per patient

- Sum central estimates by patient + questionnaire type (pre/post).
- Convert to health score.
- Pivot so each patient has pre/post on one row.
- Add improvement column (post - pre).

In [ ]:
score_sums = (
    patient_scores_df
    .groupby(['patient_id', 'questionnaire_type'], as_index=False)['central_estimate']
    .sum()
)

score_sums['health_score'] = 1 - score_sums['central_estimate']

score_pivot = (
    score_sums
    .pivot(index='patient_id', columns='questionnaire_type', values='health_score')
    .reset_index()
)

score_pivot['improvement'] = score_pivot['post'] - score_pivot['pre']
score_pivot.head(3)


### 4. Map patients to surgeons

We load a SQL query that joins `patients` and `surgeons` to associate each patient with their corresponding surgeon.


In [ ]:
with open("../sql/patient_surgeon_mapping.sql", "r") as file:
    mapping_query = file.read()

surgeon_map_df = pd.read_sql(mapping_query, engine)
surgeon_map_df.head(3)

In [ ]:
full_df = pd.merge(score_pivot, surgeon_map_df, on="patient_id", how="inner")
full_df.head(3)

### 5. Average Improvement per surgeon

Using the combined data (patient scores + surgeon names), we calculate the average improvement per surgeon.

In [ ]:
surgeon_ranking = (
    full_df
    .dropna(subset=["pre", "post"])
    .groupby("surgeon_name", as_index=False)["improvement"]
    .mean()
    .sort_values(by="improvement", ascending=False)
)

surgeon_ranking.head(10)

### 6. Most and least skillful surgeons

Based on the average improvement in patient health scores (calculated from EQ-5D-5L questionnaires pre- and post-operation), we find the following:

- **Most skillful surgeon**: `Han Solo`  
  - Average improvement: `0.131`

- **Least skillful surgeon**: `Yoda`  
  - Average improvement: `0.103`

These results are based solely on hip replacement treatments and improvement in patient-reported quality of life.


In [ ]:
# top and bottom surgeon from the ranked list
top_surgeon = surgeon_ranking.iloc[0]
bottom_surgeon = surgeon_ranking.iloc[-1]

print(f"Most skillful: {top_surgeon['surgeon_name']} ({top_surgeon['improvement']:.3f})")
print(f"Least skillful: {bottom_surgeon['surgeon_name']} ({bottom_surgeon['improvement']:.3f})")

In [ ]:
import matplotlib.pyplot as plt

top_10 = surgeon_ranking.head(10)
plt.figure(figsize=(10, 5))
plt.barh(top_10['surgeon_name'], top_10['improvement'])
plt.xlabel("Average Improvement Score")
plt.title("Top 10 Surgeons – Hip Replacement")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### Output files

- Final surgeon ranking as CSV.
- Notebook as HTML.

In [ ]:
surgeon_ranking.to_csv("../output/surgeon_ranking.csv", index=False)